In [50]:
import keras
import json
import numpy as np
from nltk import word_tokenize, sent_tokenize
from keras.models import Sequential, Model
from keras.layers import Bidirectional, LSTM, Convolution2D, Dense, Embedding, concatenate,TimeDistributed, add, dot, Merge, Input, multiply,Activation
from keras.optimizers import RMSprop
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [51]:
context_train = json.load(open('data/squad/context_train.json'))
query_train = json.load(open('data/squad/query_train.json'))

In [52]:
shared = json.load(open('data/squad/shared.json'))

In [53]:
word_emb_size = 100
T = len(context_train['context_idx'][0])
J = len(query_train['question_idx'][0])

In [54]:
num_classes = T

In [55]:
y_train_start, y_train_end = [], [] 
for x, y in zip(query_train['answer_start'], query_train['answer_end']):
    y_train_start.append(x[0])
    y_train_end.append(y[0])

In [56]:
y_train_start_encoded = to_categorical(y_train_start, num_classes)
y_train_end_encoded = to_categorical(y_train_end, num_classes)

In [57]:
emb_mat = np.array([shared['word2vec'][word] if word in shared['word2vec']
                        else np.random.multivariate_normal(np.zeros(word_emb_size), np.eye(word_emb_size))
                        for word in shared['all_words']])

In [58]:
model1_input = Input(shape = (T,))
model1_embed = Embedding(output_dim=100, input_dim=emb_mat.shape[0], weights = [emb_mat], trainable = False)(model1_input)
model1_lstm = Bidirectional(LSTM(256, return_sequences = True))(model1_embed)

In [59]:
model2_input = Input(shape = (J, ))
model2_embed = Embedding(output_dim=100, input_dim=emb_mat.shape[0], weights = [emb_mat], trainable = False)(model2_input)
model2_lstm = Bidirectional(LSTM(256, return_sequences = True))(model2_embed)

In [60]:
multiply_layer = dot([model1_lstm,model2_lstm], axes=2)

In [61]:
# modelling_model1 = Sequential()
modelling_bilstm_layer = Bidirectional(LSTM(256, return_sequences = True))(multiply_layer)
modelling_bilstm_layer_1 = Bidirectional(LSTM(256))(modelling_bilstm_layer)
modelling_bilstm_layer_return_sequences = Bidirectional(LSTM(256, return_sequences = True))(modelling_bilstm_layer) 

In [64]:
output_start_index = Dense(T,activation='softmax')(modelling_bilstm_layer_1)

In [65]:
output_end_index_lstm = LSTM(T)(modelling_bilstm_layer_return_sequences)
merge = add([output_end_index_lstm,output_start_index])

In [66]:
output_end_index_lstm = Activation('softmax')(merge)

In [67]:
final_model = Model(inputs=[model1_input, model2_input], outputs=[output_start_index, output_end_index_lstm])

In [68]:
final_model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])

In [69]:
final_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_5 (InputLayer)             (None, 866)           0                                            
____________________________________________________________________________________________________
input_6 (InputLayer)             (None, 60)            0                                            
____________________________________________________________________________________________________
embedding_5 (Embedding)          (None, 866, 100)      9596200     input_5[0][0]                    
____________________________________________________________________________________________________
embedding_6 (Embedding)          (None, 60, 100)       9596200     input_6[0][0]                    
___________________________________________________________________________________________

In [28]:
final_model.fit(
    [np.array(context_train['actual_context']), np.array(query_train['question_idx'])],
    [np.array(y_train_start_encoded), np.array(y_train_end_encoded)],
    batch_size=60,epochs = 1,validation_split = 0.2
)

Train on 78535 samples, validate on 19634 samples
Epoch 1/10
  180/78535 [..............................] - ETA: 704065s - loss: 13.5285 - dense_1_loss: 6.7686 - lstm_7_loss: 6.7599 - dense_1_acc: 0.0000e+00 - lstm_7_acc: 0.0000e+00

MemoryError: 
Apply node that caused the error: forall_inplace,cpu,scan_fn&grad_of_scan_fn&grad_of_scan_fn}(TensorConstant{866}, InplaceDimShuffle{1,0,2}.0, Subtensor{int64:int64:int64}.0, Elemwise{tanh}.0, Alloc.0, InplaceDimShuffle{0,2,1}.0, Elemwise{Composite{(i0 - sqr(i1))}}.0, InplaceDimShuffle{1,0,2}.0, Subtensor{int64:int64:int64}.0, Subtensor{int64:int64:int64}.0, Subtensor{int64:int64:int64}.0, Elemwise{tanh}.0, Alloc.0, InplaceDimShuffle{0,2,1}.0, Elemwise{Composite{(i0 - sqr(i1))}}.0, Subtensor{int64:int64:int64}.0, Subtensor{int64:int64:int64}.0, Subtensor{int64:int64:int64}.0, Alloc.0, Alloc.0, Alloc.0, Alloc.0, IncSubtensor{InplaceSet;:int64:}.0, DeepCopyOp.0, Alloc.0, Alloc.0, TensorConstant{1}, TensorConstant{866}, TensorConstant{866}, Subtensor{::, int64:int64:}.0, Subtensor{::, :int64:}.0, Subtensor{::, int64:int64:}.0, Subtensor{::, int64::}.0, Subtensor{::, int64:int64:}.0, Subtensor{::, :int64:}.0, Subtensor{::, int64::}.0, Subtensor{::, int64:int64:}.0, InplaceDim